In [1]:
import random

import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import pickle

import signature
import datasets
import utils

In [3]:
signature_data = []
try:
    with open("./data/tdsc/tumors/signature.pkl", "rb") as sig_file:
        signature_data = pickle.load(sig_file)
except IOError:
    for datum in tqdm(data):
            _, x,y = datum
            sig = calculate_signature(x)
            signature_data += [(sig, y)]
    with open("./data/tdsc/tumors/signature.pkl", "wb") as sig_file:
        pickle.dump(signature_data, sig_file)
    
        
for i in range(len(signature_data)):
    x, y = signature_data[i]
    num_zero_slices = 180 - len(x)
    zero_slices = [np.zeros(360, dtype=np.float32) for i in range(num_zero_slices)]
    x = x + zero_slices
    x=np.array(x, dtype=np.float32)
    y=np.array(y, dtype=np.float32)
    x = torch.tensor(x)
    y = torch.tensor(y).unsqueeze(0)
    signature_data[i] = (x,y)

In [4]:
malignant_signatures = range(0, 58)
benign_signatures = range(58, 100)
all_tumors = range(100)

test_set = random.sample(malignant_signatures, 10) + random.sample(benign_signatures, 10)
train_set = list(set(all_tumors) - set(test_set))

print(len(test_set)), 
print(len(train_set))

20
80


In [5]:
train_data = []
test_data = []

for idx in test_set:
    test_data.append(signature_data[idx])

for idx in train_set:
    train_data.append(signature_data[idx])

In [6]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, num_workers=8, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4, num_workers=8, shuffle=True, pin_memory=True)

In [7]:
class SequenceClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, device='cpu'):
        super(SequenceClassifier, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size * 180, num_classes)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)

        # Forward propagate LSTM
        out, _ = self.lstm(
            x, (h0, c0)
        )  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [9]:
num_epochs = 100
learning_rate = 10e-4
device = 'cuda'
criterion = torch.nn.CrossEntropyLoss()
model = SequenceClassifier(360, 180, 1, 2, device=device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
for epoch in range(num_epochs):
    print(f"Epoch #{epoch} training...")
    data_loop = tqdm(train_loader)
    for data in data_loop:
        
        x, y = data
        x = x.to(device).squeeze(1)
        y = y.to(device).squeeze(1).long()  
                
        predictions = model(x)
        loss = criterion(predictions, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        data_loop.set_postfix(loss=loss.item())
            
    total = 0
    num_corrects = 0
    
    for data in tqdm(train_loader):
        
        x, y = data
        x = x.to(device).squeeze(1)
        y = y.to(device).squeeze(1).long()  
        model.eval()
        
        with torch.no_grad():
        
            predictions = model(x)
            _, predictions = predictions.max(1)
            num_corrects += (predictions == y).float().sum()
            total += predictions.size(0)
        
        model.train()
        
    print(f"Epoch #{epoch} train acc is: {num_corrects}/{total}")
    
    total = 0
    num_corrects = 0
    
    for data in tqdm(test_loader):
        
        x, y = data
        x = x.to(device).squeeze(1)
        y = y.to(device).squeeze(1).long()  
        model.eval()
        
        with torch.no_grad():
        
            predictions = model(x)
            _, predictions = predictions.max(1)
            num_corrects += (predictions == y).float().sum()
            total += predictions.size(0)
        
        model.train()
            
    print(f"Epoch #{epoch} test acc is: {num_corrects}/{total}")

Epoch #0 training...


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch #0 train acc is: 68.0/80


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch #99 train acc is: 69.0/80


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch #99 test acc is: 12.0/20


## Using cat

In [10]:
class SequenceClassifier(torch.nn.Module):
    
    def __init__(self):
        super(SequenceClassifier, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(360*180, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 2),
            torch.nn.Softmax()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        return self.fc(x)

In [11]:
num_epochs = 100
learning_rate = 10e-4
device = 'cuda'
criterion = torch.nn.CrossEntropyLoss()
model = SequenceClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
for epoch in range(num_epochs):
    print(f"Epoch #{epoch} training...")
    data_loop = tqdm(train_loader)
    for data in data_loop:
        
        x, y = data
        x = x.to(device)
        y = y.to(device).squeeze(1).long()  
                
        predictions = model(x)
        loss = criterion(predictions, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        data_loop.set_postfix(loss=loss.item())
            
    total = 0
    num_corrects = 0
    
    for data in tqdm(train_loader):
        
        x, y = data
        x = x.to(device)
        y = y.to(device).squeeze(1).long()  
        model.eval()
        
        with torch.no_grad():
        
            predictions = model(x)
            _, predictions = predictions.max(1)
            num_corrects += (predictions == y).float().sum()
            total += predictions.size(0)
        
        model.train()
        
    print(f"Epoch #{epoch} train acc is: {(num_corrects/total)*100}")
    
    for data in tqdm(test_loader):
        
        x, y = data
        x = x.to(device)
        y = y.to(device).squeeze(1).long()  
        model.eval()
        
        with torch.no_grad():
        
            predictions = model(x)
            _, predictions = predictions.max(1)
            num_corrects += (predictions == y).float().sum()
            total += predictions.size(0)
        
        model.train()
            
    print(f"Epoch #{epoch} test acc is: {(num_corrects/total)*100}")

Epoch #0 training...


  0%|          | 0/20 [00:00<?, ?it/s]

/home/ozma/anaconda3/envs/torch/lib/python3.9/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch #0 train acc is: 60.000003814697266


...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch #99 train acc is: 60.000003814697266


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch #99 test acc is: 58.0
